In [2]:
import asyncio
from pyppeteer import launch
from json import loads
from keyboard import KeyboardAction

def _patch_pyppeteer():
    from typing import Any
    from pyppeteer import connection, launcher
    import websockets.client

    class PatchedConnection(connection.Connection):  # type: ignore
        def __init__(self, *args: Any, **kwargs: Any) -> None:
            super().__init__(*args, **kwargs)
            # the _ws argument is not yet connected, can simply be replaced with another
            # with better defaults.
            self._ws = websockets.client.connect(
                self._url,
                loop=self._loop,
                # the following parameters are all passed to WebSocketCommonProtocol
                # which markes all three as Optional, but connect() doesn't, hence the liberal
                # use of type: ignore on these lines.
                # fixed upstream but not yet released, see aaugustin/websockets#93ad88
                max_size=None,  # type: ignore
                ping_interval=None,  # type: ignore
                ping_timeout=None,  # type: ignore
            )

    connection.Connection = PatchedConnection
    # also imported as a  global in pyppeteer.launcher
    launcher.Connection = PatchedConnection

_patch_pyppeteer()

In [3]:
browser = await launch(headless=False)
page = await browser.newPage()
await page.goto('https://scorecounter.com/click-counter/')
await page.evaluate('''
        let keyEvents = [];
        let keySensorInstallDate = +new Date();
        ['keyup', 'keydown'].forEach(k =>
            window.addEventListener(k, 
                e => keyEvents.push({
                    event: k, 
                    key: String.fromCharCode(e.keyCode),
                    time: +new Date() - keySensorInstallDate
                }), 
            true)
        );
        window._getKeyboardEvents = () => keyEvents.splice(0, keyEvents.length);
        ''', force_expr=True)
# client = await page.target().createCDPSession()
# await client.send()

{}

In [6]:
# q_count = 0
# while(q_count < 3):
res = await page.evaluate("JSON.stringify(window._getKeyboardEvents());")
keys = loads(res)
keyboard_actions = [(k['time'], KeyboardAction(k["event"], k["key"])) for k in keys]
keyboard_actions

[(36839, {'key': 'T', 'keycode': 84, 'eventtype': 'keydown'}),
 (36951, {'key': 'E', 'keycode': 69, 'eventtype': 'keydown'}),
 (37003, {'key': 'T', 'keycode': 84, 'eventtype': 'keyup'}),
 (37026, {'key': 'E', 'keycode': 69, 'eventtype': 'keyup'}),
 (37135, {'key': 'S', 'keycode': 83, 'eventtype': 'keydown'}),
 (37211, {'key': 'S', 'keycode': 83, 'eventtype': 'keyup'}),
 (37259, {'key': 'T', 'keycode': 84, 'eventtype': 'keydown'}),
 (37340, {'key': 'T', 'keycode': 84, 'eventtype': 'keyup'})]

In [7]:
await browser.close()

In [3]:
asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running